# temp BA-288 Список оформленных заказов

## magic

In [67]:
# magic
############################################################
# magic не запускаются с посмощью run !! поэтому отдельно будут тут! # %magic # посмотреть список magic команд
%matplotlib inline

# run files
%run /srv/jhub/share/._Additions/Import_libs.py
%run /srv/jhub/share/._Additions/Connecting_bi_admin.py
%run /srv/jhub/share/._Additions/my_script.py
%run /srv/jhub/share/._Additions/Date_gen.py
%run /srv/jhub/share/._Additions/xx.py

# print
############################################################
print(
df_test.shape, '\n'
,abs_path, '\n' # abs_path - путь до шары (видна всем езерам - '/srv/jhub/share/')
,path_OUT_dbDATA, '\n'
,date_start, date_end, '\n'
)
############################## End ##############################

(1, 1) 
 /home/slebedev/jhub-analytics/ 
 /home/slebedev/jhub-analytics/Аналитика/_DATA/ 
 2017-01-01 2021-11-01 



In [68]:
# выводим спарсенные данные с жиры описание и тд....
task_link = '''
https://jira.e2e4.ru/browse/BA-\
288
'''
tasks(loginAtlassianjhubAdmin, passwordAtlassianjhubAdmin, task_link)

Бизнес аналитика / BA-288 

    Обновить отчет по оформленным заказам с сайта 


    Описание: 
    
Дано
У нас есть отчет со списком оформленных заказов с сайта: https://confluence.e2e4.ru/pages/viewpage.action?pageId=183326070&run_1_date=11.04.2019&run_1=run
Как мы его используем:

для сверки Яндекс.Метрики и Google Analytics
для сверки систем рекомендации и триггерных писем

Сейчас он нам нужен для проверки результатов работы CPA-сетей. Там мы будем платить за каждый оплаченный заказ. 
Мы хотим выгружать его с периодичностью (например, раз в месяц) и сверять с данными CPA-сетей.
Для этого нам нужны:

дата и время создания,
клиент,
номер заказа,
статус заказа (сейчас там нет ничего про оплату),
сумма заказа.

Но в существующем отчете действует ограничение в 10 000 строк, поэтому мы не можем выгрузить ничего более 6–7 дней.
Что нужно

Проверить отчет на адекватность.
Добавить информацию, оплачен ли заказ.
Перенести его в Excel-выгрузку.

И бонусный вопрос: какой статус и сумма будет у

## Путь куда будут выгружены файлы¶

In [29]:
folder_out = '_temp/'
file_name = 'BA-288 Список оформленных заказов'

path_OUT_dbDATA + folder_out + file_name

'/home/slebedev/jhub-analytics/Аналитика/_DATA/_temp/BA-288 Список оформленных заказов'

## Качнуть за какую то опредлеленную дату

In [62]:
#  Качнуть за какую то опредлеленную дату
date_start = '01.10.2021'
date_end = '31.10.2021'

path_OUT_dbDATA + folder_out + file_name + fr' { date_start } { date_end }.csv'

'/home/slebedev/jhub-analytics/Аналитика/_DATA/_temp/BA-288 Список оформленных заказов 01.10.2021 31.10.2021.csv'

## SQL

In [63]:
# DATE_START_replce date_start
# DATE_END_replce date_end

sql_288 = '''     
         /*
         * BA-288 Список оформленных заказов
         * https://confluence.e2e4.ru/pages/viewpage.action?pageId=183326070
         * https://jira.e2e4.ru/browse/BA-288
         * */

        with params as materialized (
        select 
    --		('01.09.2021' || ' 00:00:00')::timestamp date_start --32 614 тут и  32 614 o2
    --		,('30.09.2021' || ' 23:59:59')::timestamp date_end
            -- ( '\$frm' || ' 00:00:00')::timestamp date_start 
            -- ,( '\$til' || ' 23:59:59')::timestamp date_end 
            ( 'DATE_START_replce' || ' 00:00:00')::timestamp    date_start 
            ,( 'DATE_END_replce' || ' 23:59:59')::timestamp    date_end
        )

        , payments_1 as (
        select distinct
            ai.tm_created "Дата создания заказа" --время Дата создания заказа
        -- 	,concat( '<a href="http://opentech.local/new/order/',a.id,'?main=2&additional=8">',a.id,'</a>' ) "ID заказа" --позиции клиентских заказов
            ,case when a.id is null then null else 
                concat( '=HYPERLINK("http://opentech.local/new/order/',a.id,'", "',a.id,'")' ) end "ID заказа" --excel
            --,a.id "ID заказа" --позиции клиентских заказов
            ,state.name "Cтатус заказа" --справочник состояний клиентского заказа
            ,c.name "Клиент (Покупатель)" 
        --	,concat('<a href="http://opentech.local/index.php?p=customers&id=',c.id,']">',c.name,'</a>') "Клиент (Покупатель)"
    --        ,case when c.id is null then null else 
    --            concat( '=HYPERLINK("http://opentech.local/index.php?p=customers&id=',c.id,'", "',c.name,'")' ) end "Клиент (Покупатель)" --excel	
                /*кавычки внутри фирмы не понимает и вываливает 0 сучий ексель!*/

            --,c.name "Клиент c.name"
            ,round(orders.get_order_summa_brutto(a.id),2) "Стоимость оформленных позиций" --вычисление общей суммы заказа COALESCE(SUM(a.price_net), 0)
            ,pa.payment_id "pa.payment_id"
            ,pa.sum "pa.sum"
        from params _
        cross join opentech.account_item ai
        join orders.states state on state.id = ai.state
        join opentech.accounts a on a.id = ai.account_id
        join opentech.customers c on c.id=a.id_customer
        left join opentech.payments_account pa on pa.account_id = a.id
        where 
        --	case when 666=\$ordersatate then true else state.id=\$ordersatate end
        --	and ai.tm_created::date between '\$frm' and '\$til' --время Дата создания заказаия заказа
        --	case when 0=0 then true else state.id=2 end
        --	and 
            ai.tm_created::date between _.date_start and _.date_end
            and
            ai.siteuserid is not null --учетная запись сайта, с которой был оформлен заказ
            and a.user_login = 'client' --кто добавил позицию в заказ
    --		and a.state = 23 --по стат продано (Состояние позиции заказа) --которые купили товары или услуги в статусе списания "Продано"

    --		and 
    --    	a.id in ( 6248305, 6255767 ) --"ID заказа" 6255767

        )

        , orders_1 as (
        select --distinct
        -- 	concat('<a href="http://opentech.local/new/order/',a.id,'?main=2&additional=8">',a.id,'</a>') "ID заказа" --позиции клиентских заказов
            case when a.id is null then null else 
                concat( '=HYPERLINK("http://opentech.local/new/order/',a.id,'", "',a.id,'")' ) end "ID заказа" --excel
            ,sum(a.price_net) "Стоимость отгруженных позиций" --сумма всех позиц (продажная) в статусе продано 
        from params _
        cross join opentech.account_item ai
        join orders.states state on state.id = ai.state
        join opentech.accounts a on a.id = ai.account_id
        --left join opentech.store os ON a.store_id = os.id
        /* место продажи */
        join opentech.storage storage on storage.id = a.storage_id
        join public.cities pc on pc.id = storage.city_id 
        where 
        --	case when 0=\$city then true else pc.id=\$city end
        --	and case when 0=\$ordersatate then true else state.id=\$ordersatate end
        --	and ai.tm_created::date between '\$frm' and '\$til' --время Дата создания заказаия заказа
        --	case when 0=0 then true else pc.id=659 end
        --	and case when 0=0 then true else state.id=2 end
        --	and 
    --        ai.tm_created::date between _.date_start and _.date_end
    --        and 
            ai.siteuserid is not null --учетная запись сайта, с которой был оформлен заказ 
            and a.user_login = 'client' --кто добавил позицию в заказ
            /* "Стоимость отгруженных позиций" поэтому a.state = 23*/
            and a.state = 23 --по стат продано (Состояние позиции заказа) --которые купили товары или услуги в статусе списания "Продано"

    --		and 
    --    	a.id in ( 6248305, 6255767 ) --"ID заказа" 6255767

        group by 1

        )

        --select * from payments_1
        , orders_2 as (
        select distinct
            ai.tm_created "Дата создания заказа" --время Дата создания заказаия заказа
        -- 	,concat('<a href="http://opentech.local/new/order/',a.id,'?main=2&additional=8">',a.id,'</a>') "ID заказа" --позиции клиентских заказов
            ,case when a.id is null then null else 
                concat( '=HYPERLINK("http://opentech.local/new/order/',a.id,'", "',a.id,'")' ) end "ID заказа" --excel
        --	,a.id "ID заказа 2" --позиции клиентских заказов
            ,state.name "Cтатус заказа" --справочник состояний клиентского заказа
            ,c.name "Клиент (Покупатель)" 
        --	,concat('<a href="http://opentech.local/index.php?p=customers&id=',c.id,']">',c.name,'</a>') "Клиент (Покупатель)"
        --  	,case when c.id is null then null else 
        --      concat( '=HYPERLINK("http://opentech.local/index.php?p=customers&id=',c.id,'", "',c.name,'")' ) end "Клиент (Покупатель)" --excel

        --	,c.id "ID клиента"
            ,pc.name "Город отгрузки"
            ,sum(a.price_net) "Стоимость оформленных позиций"
        --	,round(orders.get_order_summa_brutto(a.id),2) "Стоимость оформленных позиций" --вычисление общей суммы заказа
            ,sum(os.sprice) "Чистая себестоимость позиций"
            ,case when sum(os.sprice) is null then null
                else sum(a.price_net) - sum(os.sprice) end "Валовый доход по ЧС, руб" -- если нет чистой себестоимости, то выводим доход от расчетной себестоимости (доход: цена продажи − чистая себестоимость)
        --	,ai.id_platform "id_platform" -- Сайт старый новый, Маркетплейс, Платформа, Площадка
            ,op.name "Площадка"

            /*не совсем верно надо переделать или убрать согласно новым данным с op.name "Площадка" (переделать на id)*/
        --	,case 
        --        when ai.siteuserid is null then 'менеджер' --Учетная запись сайта, с которой был оформлен заказ
        --        when (a.user_login ilike 'SHOP') and (op.name ilike '20?') then 'новый сайт' 
        --        when (a.user_login ilike 'SHOP') and (op.name ilike 'Cайт') then 'старый сайт' 
        --        	else 'маркетплейсы' end as "Кто оформил заказ"

        from params _
        cross join opentech.account_item ai
        join orders.states state on state.id = ai.state
        join opentech.accounts a on a.id = ai.account_id
        join opentech.customers c on c.id=a.id_customer
        left join opentech.store os ON a.store_id = os.id
        /* место продажи */
        join opentech.storage storage on storage.id = a.storage_id
        join public.cities pc on pc.id = storage.city_id 
        /*Сайт старый новый, Маркетплейс, Платформа, Площадка*/
        left join orders.platforms op on op.id = ai.id_platform
        where 
        --	case when 0=\$city then true else pc.id=\$city end
        --	and case when 666=\$ordersatate then true else state.id=\$ordersatate end
        --	and case when 0=\$ordersplatform then true else op.id=\$ordersplatform end
        --	and ai.tm_created::date between '\$frm' and '\$til' --время Дата создания заказаия заказа

        --	case when 0=0 then true else pc.id=659 end
        --	and case when 0=0 then true else state.id=2 end
        --	and case when 0=0 then true else op.id=3252 end --Сайт-20??
        --	and 
            ai.tm_created::date between _.date_start and _.date_end
            and 
            ai.siteuserid is not null --учетная запись сайта, с которой был оформлен заказ
            and a.user_login = 'client' --кто добавил позицию в заказ 

    --		and 
    --    	a.id in ( 6248305, 6255767 ) --"ID заказа" 6255767
            --	and op.name = 'Сайт-2009'

        group by 2,1,3,4,5,9 --,10

        )

        --select * from orders_2
        , end_all as materialized (
        select 
            --row_number() OVER()::text "№"
            o2."Город отгрузки"
            ,o2."Дата создания заказа"::text
            ,o2."ID заказа"::text --,p1."ID заказа 2"::text, 
            ,o2."Cтатус заказа"::text
            ,o2."Клиент (Покупатель)"::text --p1."ID клиента"--,p1."Клиент просто номер"
            ,array_to_string(array_agg(p1."pa.payment_id"),' ') "ID платежей"

    --		,array_to_string(array_agg(concat(
    --			'<a href="http://opentech.local/payments/',p1."pa.payment_id",'">',p1."pa.payment_id",'</a>'
    --			)),'; ') "ID платежей"

    --        ,case when max(p1."pa.payment_id") is null then null else 
    --            array_to_string(array_agg(concat(
    --            '=HYPERLINK("http://opentech.local/payments/',p1."pa.payment_id",'", "',p1."pa.payment_id",'")' 
    --            )),' ') end "ID платежей" --excel

            ,array_to_string(array_agg( round(p1."pa.sum",2)),' ') "Платежи по заказу"
            ,sum(p1."pa.sum") "Оплачено по заказу"
            ,p1."Стоимость оформленных позиций"
            ,o1."Стоимость отгруженных позиций"
            ,o2."Чистая себестоимость позиций"
            /*деление на ноль*/
            ,case when p1."Стоимость оформленных позиций" <> 0 then round((sum(p1."pa.sum") / p1."Стоимость оформленных позиций")*100,0)
                else null end "% выкупа по доходу"
            ,o2."Валовый доход по ЧС, руб"
            ,o2."Площадка"
        --	,o2."Кто оформил заказ"

        from orders_2 o2
        left join orders_1 o1 on o1."ID заказа" = o2."ID заказа"
        left join payments_1 p1 on p1."ID заказа" = o2."ID заказа" 
    --        and p1."Дата создания заказа" = o2."Дата создания заказа"  
    --        and p1."Cтатус заказа" = o2."Cтатус заказа" 
    --        and p1."Клиент (Покупатель)" = o2."Клиент (Покупатель)" 
    --        and p1."Стоимость оформленных позиций" = o2."Стоимость оформленных позиций"

        group by 
        --	o2."ID заказа",o2."Дата создания заказа",o2."Город отгрузки",o2."Cтатус заказа",o2."Клиент (Покупатель)",o2."pa.payment_id",o2."Стоимость оформленных позиций"
        --	,"Стоимость отгруженных позиций","Чистая себестоимость позиций","Валовый доход по ЧС, руб","Площадка" 
            3,2,1,4,5 ,9,10 ,11,13,14 --,15
        order by o2."ID заказа" --7--2
        )

        select --distinct 
        --	row_number() OVER()::text "№", 
            * 
        from end_all  

        union all
        select 
        --	null "№",
            null "Город отгрузки"
            ,null "Дата создания заказа"
            ,(select distinct count(ea."ID заказа") from end_all ea)::text "ID заказа"
            ,null "Cтатус заказа"
            ,(select distinct count(ea."Клиент (Покупатель)") from end_all ea)::text "Клиент (Покупатель)"
            ,null "ID платежей" 
            ,null "Платежи по заказу"
            ,(select sum(ea."Оплачено по заказу") from end_all ea) "Оплачено по заказу"
            ,(select sum(ea."Стоимость оформленных позиций") from end_all ea) "Стоимость оформленных позиций"	
            ,null "Стоимость отгруженных позиций"
        ,null,null,null,null--,null
'''


## коннектимся, подставляем даты, сохраняем в файл

In [64]:
df_fon = pd.DataFrame()
#   коннектимся
with psycopg2.connect(dbname=database, user=user, password=password, host=host) as cnxn:
#   , подставляем даты
    df_fon = pd.read_sql_query( sql_288.replace('DATE_START_replce', f'{ date_start }').replace('DATE_END_replce', f'{ date_end }')
                               ,cnxn)
    
# to_csv
# df_fon.to_csv(
#     path_OUT_dbDATA + folder_out + file_name + fr' { date_start } { date_end }.csv' \
#     , sep='☺',  index=True) # cp-1251 encoding='UTF-8', engine='xlsxwriter'

# to_excel
df_fon.to_excel(
    path_OUT_dbDATA + folder_out + file_name + fr' { date_start } { date_end }.xlsx' \
    , index=False ) # cp-1251 encoding='UTF-8', engine='xlsxwriter'

print(df_fon, '\n\n', date_start, date_end)
print('\n\n\n*** End ***\n\n\n')

      Город отгрузки           Дата создания заказа  \
0            Северск  2021-10-01 00:01:58.895434+07   
1        Новокузнецк  2021-10-01 00:04:49.671716+07   
2        Новосибирск  2021-10-01 00:06:13.057852+07   
3             Тюмень  2021-10-01 00:06:16.522711+07   
4            Барнаул   2021-10-01 00:07:50.85878+07   
...              ...                            ...   
27541    Новосибирск  2021-10-29 18:09:09.951061+07   
27542           Омск  2021-10-29 18:09:41.625843+07   
27543        Барнаул  2021-10-29 18:15:19.315438+07   
27544      Хабаровск  2021-10-29 18:17:05.418128+07   
27545           None                           None   

                                               ID заказа Cтатус заказа  \
0      =HYPERLINK("http://opentech.local/new/order/63...   Не оформлен   
1      =HYPERLINK("http://opentech.local/new/order/63...   Не оформлен   
2      =HYPERLINK("http://opentech.local/new/order/63...         Выдан   
3      =HYPERLINK("http://opentech.local/ne

In [65]:
# sql_288.replace('DATE_START_replce', f'{ date_start }').replace('DATE_END_replce', f'{ date_end }')

In [61]:
# # to_excel
# df_fon.to_excel(
#     path_OUT_dbDATA + folder_out + file_name + fr' { date_start } { date_end }.xlsx' \
#     , index=False ) # cp-1251 encoding='UTF-8', engine='xlsxwriter'

# print(df_fon, '\n\n', date_start, date_end)
# print('\n\n\n*** End ***\n\n\n')